# Entrainement du modéle

### Importation des bibliotéque

In [1]:
import math
import cv2
import numpy as np
import pandas as pd
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
from IPython.display import HTML
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

### Importation et préparation des données

In [2]:
# Importer le fichier CSV dans un DataFrame
df = pd.read_csv("df_label.csv")

# Afficher les premières lignes du DataFrame
df.head()


,Angles_Elbow_Right,Angles_Elbow_Left,Angles_Shoulder_Right,Angles_Shoulder_Left,Angles_Hip_Right,Angles_Hip_Left,Angles_Wrist_Right,Angles_Wrist_Left,Angles_Shoulder_lateral_Left,Angles_Shoulder_lateral_right,Angles_Shoulder_lateral2_Left,Angles_Shoulder_lateral2_right,Angles_Hand_Left,Angles_Hand_right,Angles_Hand_lateral_Left,Angles_Hand_lateral_right,y
0,352.376927,39.152745,24.839147,336.654340,336.211505,21.406423,106.260205,135.000000,319.502005,43.311782,259.161253,109.676866,1.005086,285.255119,226.005086,178.994914,Start_position
1,254.505151,41.764501,23.953761,338.116308,337.227003,22.884949,225.000000,184.398705,319.700399,43.689828,261.867933,106.656091,45.400663,308.290163,221.001958,83.290163,Start_position
2,352.541872,25.129966,25.927140,337.917626,336.765818,18.992988,33.690068,217.303948,317.860786,46.754280,258.524311,108.386094,54.352979,205.346176,197.049031,171.656108,Start_position
3,354.260902,23.481391,25.628984,337.307757,337.441869,18.937620,77.471192,244.653824,316.945483,46.552389,257.828542,108.267906,84.936383,255.963757,200.282559,178.492564,Start_position
4,341.896237,37.658774,25.261576,338.973627,338.366838,16.831344,206.565051,233.972627,318.732757,45.818758,258.906277,108.103763,89.068443,9.039483,215.095817,162.474432,Start_position


🧩 Ensuite, les données sont divisées en deux ensembles : un ensemble d'entraînement (X_train et y_train) et un ensemble de test (X_test et y_test). L'ensemble de test sera utilisé pour évaluer la performance du modèle après l'entraînement.

🔢 Les étiquettes dans y_train et y_test sont converties en valeurs numériques à l'aide d'un encodeur (LabelEncoder). Cela signifie que les catégories ou les classes sont représentées par des nombres. Par exemple, si vous aviez des étiquettes "Chat" et "Chien", elles pourraient être transformées en 0 et 1.

🌟 Enfin, les étiquettes encodées sont converties en "catégories one-hot". 

In [21]:
X = df.drop(columns='y')
y = df[['y']]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encoder les étiquettes en valeurs numériques
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convertir les valeurs numériques en catégories one-hot
num_classes = len(label_encoder.classes_)
y_train_encoded = tf.keras.utils.to_categorical(y_train_encoded, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test_encoded, num_classes)

In [6]:
X_train.shape, X_test.shape, y_train_encoded.shape, y_test_encoded.shape

((3496, 16), (874, 16), (3496, 68), (874, 68))

## CNN

🌐 Les CNN sont particulièrement adaptés à l'analyse de données spatiales, telles que les informations provenant de capteurs d'angle corporel. Ces réseaux sont conçus pour extraire automatiquement des caractéristiques pertinentes des données d'entrée, ce qui les rend adaptés à la détection de motifs complexes et subtils dans les séquences d'angles articulaires.

📏 Les CNN sont capables de gérer des données de grande dimension, ce qui est souvent le cas avec les données d'angle corporel collectées à partir de capteurs multiples.

🕒 Les CNN sont intrinsèquement adaptés à la détection de mouvement, car ils sont capables de prendre en compte la séquentialité des données. Ils peuvent apprendre des modèles de mouvement à partir de séquences temporelles d'angles corporels, ce qui permet une classification précise des différentes actions ou mouvements.

🔍 En outre, l'utilisation de CNN dans ce contexte peut permettre une réduction significative de la nécessité de prétraitement manuel des données, car ces réseaux peuvent apprendre à extraire des caractéristiques pertinentes directement à partir des données brutes.

In [8]:
def initialize_model():
    num_angles=16
    
    # Définir l'architecture du CNN
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(num_angles,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(80, activation='relu'))
    model.add(layers.Dense(68, activation='softmax'))
    
    # Compiler le modèle
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
# Sauvegarder le LabelEncoder dans un fichier
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


In [10]:
model = initialize_model()

# Entraîner le modèle
model.fit(X_train, y_train_encoded, validation_split=0.3, epochs=150)

Epoch 1/150
77/77 [==============================] - 1s 4ms/step - loss: 8.4408 - accuracy: 0.2342 - val_loss: 3.1099 - val_accuracy: 0.3451
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 2.7911 - accuracy: 0.4246 - val_loss: 2.6557 - val_accuracy: 0.4500
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 2.3915 - accuracy: 0.4810 - val_loss: 2.5500 - val_accuracy: 0.3899
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 2.1507 - accuracy: 0.4986 - val_loss: 2.1352 - val_accuracy: 0.4795
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 1.9783 - accuracy: 0.5317 - val_loss: 1.9948 - val_accuracy: 0.5224
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 1.8523 - accuracy: 0.5533 - val_loss: 1.9855 - val_accuracy: 0.5338
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 1.7435 - accuracy: 0.5631 - val_loss: 1.8787 - val_accuracy: 0.5167
Epoch 8/150
7

Epoch 59/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4464 - accuracy: 0.8492 - val_loss: 1.3475 - val_accuracy: 0.7684
Epoch 60/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4565 - accuracy: 0.8410 - val_loss: 1.3955 - val_accuracy: 0.7741
Epoch 61/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4234 - accuracy: 0.8647 - val_loss: 1.3753 - val_accuracy: 0.7674
Epoch 62/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4074 - accuracy: 0.8627 - val_loss: 1.4448 - val_accuracy: 0.7502
Epoch 63/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4233 - accuracy: 0.8488 - val_loss: 1.4653 - val_accuracy: 0.7378
Epoch 64/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4718 - accuracy: 0.8394 - val_loss: 1.3713 - val_accuracy: 0.7769
Epoch 65/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4028 - accuracy: 0.8635 - val_loss: 1.3982 - val_accuracy: 0.7712
Epoch 

77/77 [==============================] - 0s 2ms/step - loss: 0.2432 - accuracy: 0.9166 - val_loss: 1.6971 - val_accuracy: 0.8017
Epoch 117/150
77/77 [==============================] - 0s 2ms/step - loss: 0.2182 - accuracy: 0.9170 - val_loss: 1.6773 - val_accuracy: 0.8246
Epoch 118/150
77/77 [==============================] - 0s 2ms/step - loss: 0.1934 - accuracy: 0.9326 - val_loss: 1.6597 - val_accuracy: 0.8284
Epoch 119/150
77/77 [==============================] - 0s 2ms/step - loss: 0.2071 - accuracy: 0.9244 - val_loss: 1.6709 - val_accuracy: 0.8093
Epoch 120/150
77/77 [==============================] - 0s 2ms/step - loss: 0.2040 - accuracy: 0.9224 - val_loss: 1.7914 - val_accuracy: 0.8103
Epoch 121/150
77/77 [==============================] - 0s 2ms/step - loss: 0.2278 - accuracy: 0.9224 - val_loss: 1.8207 - val_accuracy: 0.7950
Epoch 122/150
77/77 [==============================] - 0s 2ms/step - loss: 0.2253 - accuracy: 0.9158 - val_loss: 1.8530 - val_accuracy: 0.7960
Epoch 123/150

In [11]:
# Évaluer le modèle
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test accuracy:', test_acc)

28/28 [==============================] - 0s 1ms/step - loss: 2.2798 - accuracy: 0.7803
Test accuracy: 0.7803203463554382


In [12]:
y_test_encoded

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
# Prédire de nouvelles positions
predictions = model.predict(X_test[2:3])

# Traiter les cas inconnus
threshold = 0.5  # Seuil pour déterminer si une position est inconnue
unknown_indices = np.where(np.max(predictions, axis=1) < threshold)[0]
for i in unknown_indices:
    predictions[i] = np.zeros(num_classes)  # Réinitialiser la prédiction à zéro
    predictions[i, unknown_pose_label_index] = 1  # Attribuer le label de la position inconnue

1/1 [==============================] - 0s 76ms/step


In [14]:
# Sauvegarder le modèle
model.save('CNN1.h5')